In [1]:
import mne
import numpy as np
print(mne.__version__)
print(np.__version__)
import sys
sys.path.append('../src/utils/data_preprocessing')
sys.path.append('../src/model')

import temp_utils as utils
from  svm import train_svm_model

1.6.0
1.26.1


In [2]:
file_path = r'../src/data/test_data/1/1.edf'
raw = utils.read_edf(file_path)
raw = utils.rename_edf(raw)
with mne.utils.use_log_level('WARNING'):
        raw.filter(1, 60, fir_design='firwin')

picks = utils.obtain_picks(raw)
info = utils.obtain_eeg_info(raw, picks)

Extracting EDF parameters from /Users/jie/dev/EEG_DL/src/data/test_data/1/1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1799999  =      0.000 ...  3599.998 secs...


/Users/jie/dev/EEG_DL/notebooks/../src/utils/data_preprocessing/temp_utils.py:12: RuntimeWarning: Scaling factor is not defined in following channels:
POL SpO2
  raw = mne.io.read_raw_edf(file_path, preload=True, encoding='latin1')
/Users/jie/dev/EEG_DL/notebooks/../src/utils/data_preprocessing/temp_utils.py:12: RuntimeWarning: Physical range is not defined in following channels:
POL SpO2
  raw = mne.io.read_raw_edf(file_path, preload=True, encoding='latin1')


In [4]:
utils.obtain_multi_topomap(raw, picks, info, '1', fig_size=128, is_energy=True, is_test=False)

Jie Log: output_dir: /Users/jie/dev/EEG_DL/src/data/output_image/1


Obtain topomap for 1: 100%|██████████| 3600/3600 [04:54<00:00, 12.20it/s]


In [ ]:
from PIL import Image
import os

# 文件夹路径
main_path = r'D:\Research\EEG\EEG_DL_Classifier\tests'
eeg_file_name = '1'
folder_path = os.path.join(main_path, eeg_file_name)

output_path = r'D:\Research\EEG\EEG_DL_Classifier\tests\eeg.gif'


# 读取所有图像文件
images = []
for i in range(3600):
    file_name = f'{eeg_file_name}_{i}.png'  # 假设文件命名方式为 "0.png", "1.png", ..., "3599.png"
    file_path = os.path.join(folder_path, file_name)
    if os.path.exists(file_path):
        img = Image.open(file_path)
        images.append(img)




In [ ]:
gif_path = output_path
images[0].save(gif_path, save_all=True, append_images=images[0:], optimize=False, duration=100, loop=1)

In [2]:
egg_filename = '1'
X_train, X_test, y_train, y_test = utils.prepare_data(egg_filename, is_test = False)

Jie log: images_np.shape: (3600, 16384), labels.shape: (3600,)


In [10]:
print(f"训练集中的0：{sum(y_train==0)/len(y_train)}")
print(f"训练集中的1：{sum(y_train==1)/len(y_train)}")
print('\n')
print(f"测试集中的0：{sum(y_test==0)/len(y_test)}")
print(f"测试集中的1：{sum(y_test==1)/len(y_test)}")

训练集中的0：0.11706349206349206
训练集中的1：0.8829365079365079


测试集中的0：0.09537037037037037
测试集中的1：0.9046296296296297


In [3]:
train_svm_model(X_train, y_train, X_test, y_test)

Training SVM model...
X_train shape: (2520, 16384)
y_train shape: (2520,)
X_test shape: (1080, 16384)
y_test shape: (1080,)
Accuracy: 0.9046296296296297
Recall: 1.0
Precision: 0.9046296296296297
F1 Score: 0.9499270782693243
Confusion Matrix:
[[  0 103]
 [  0 977]]


SVC()